In [41]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
import cv2
from PIL import Image

# Step 1: Set Paths
current_path = os.getcwd()
image_dir = os.path.join(current_path, 'New Plant Diseases Dataset(Augmented)', 
                         'New Plant Diseases Dataset(Augmented)', 'train')

# Step 2: Show total images in one folder
print("Total Tomato___Bacterial_spot images:", 
      len(os.listdir(os.path.join(image_dir, 'Tomato___Bacterial_spot'))))

# Step 3: Load file paths into train_dict
train_dict = {dir: [] for dir in os.listdir(image_dir)}
for dir in os.listdir(image_dir):
    directory_path = os.path.join(image_dir, dir)
    for image in os.listdir(directory_path):
        train_dict[dir].append(os.path.join(directory_path, image))

# Step 4: View sample image (optional)
Image.open(train_dict['Tomato___Bacterial_spot'][0])

# Step 5: Load images and labels efficiently
img_arr_lst = []
labels = []
MAX_IMAGES_PER_CLASS = 100  # 🚀 Load fewer images to speed up training

for key, val in train_dict.items():
    for i, img in enumerate(val):
        if i >= MAX_IMAGES_PER_CLASS:
            break
        try:
            img_arr = cv2.imread(img)
            if img_arr is None:
                continue
            resized_img = cv2.resize(img_arr, (224, 224))  # needed for model input
            img_arr_lst.append(resized_img)
            labels.append(key)
        except Exception as e:
            print(f"Skipped: {img} | Reason: {e}")

# Step 6: Convert to NumPy arrays
img_arr_lst = np.array(img_arr_lst)
labels = np.array(labels)

# Step 7: Convert class names to integer labels
label_to_index = {label: idx for idx, label in enumerate(train_dict.keys())}
labels = np.array([label_to_index[label] for label in labels])
num_classes = len(label_to_index)

# Step 8: Train/test split
x_train, x_test, y_train, y_test = train_test_split(img_arr_lst, labels, test_size=0.2)
x_train = x_train / 255.0
x_test = x_test / 255.0

# Step 9: Define CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')  # 🧠 Multi-class output
])

# Step 10: Compile and Train Model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=32)


Total Tomato___Bacterial_spot images: 1702
Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 36s 355ms/step - accuracy: 0.0648 - loss: 5.0068
Epoch 2/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.4458 - loss: 2.0545
Epoch 3/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 34s 361ms/step - accuracy: 0.7850 - loss: 0.7974
Epoch 4/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 94s 999ms/step - accuracy: 0.9281 - loss: 0.2737
Epoch 5/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 210s 2s/step - accuracy: 0.9740 - loss: 0.1229
Epoch 6/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 163s 2s/step - accuracy: 0.9821 - loss: 0.0621
Epoch 7/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 61s 629ms/step - accuracy: 0.9938 - loss: 0.0323
Epoch 8/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 33s 342ms/step - accuracy: 0.9831 - loss: 0.0778
Epoch 9/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 34s 358ms/step - accuracy: 0.9927 - loss: 0.0346
Epoch 10/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 35s 365ms/step - accuracy: 0.9970 - loss: 0.0112


In [42]:
pred=model.predict(x_test)
pred

24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step


array([[1.95937474e-07, 2.73135098e-10, 2.06177570e-02, ...,
        6.57207178e-10, 6.53124236e-15, 6.33758085e-04],
       [3.28242140e-06, 1.22501604e-01, 2.72848483e-05, ...,
        4.04062775e-05, 3.97803269e-05, 8.26250698e-07],
       [3.07504351e-08, 1.59688655e-03, 8.54702322e-13, ...,
        3.03713523e-06, 2.01641953e-13, 5.57505498e-14],
       ...,
       [2.97584866e-05, 3.34332326e-05, 3.86883954e-07, ...,
        3.97889926e-05, 8.08721306e-05, 4.50039588e-05],
       [1.47486219e-06, 5.36060618e-09, 6.34777143e-06, ...,
        2.79597769e-11, 2.32589358e-17, 1.64312883e-10],
       [1.03468352e-04, 4.73288651e-07, 1.70180851e-04, ...,
        1.10812206e-03, 7.83428167e-09, 7.26033642e-04]], dtype=float32)

In [43]:
pred=(pred>0.5).astype(int).ravel()
pred

array([0, 0, 0, ..., 0, 0, 0])

In [44]:
y_test

array([15,  1, 19, 20, 34, 29, 29, 20, 31,  8, 22, 31,  8, 22,  4, 18, 27,
        9, 17, 14,  9, 18, 31, 14,  0,  9, 21, 10,  0,  7, 34, 10,  4, 19,
       24, 27, 13,  3, 35, 27, 27,  1, 35, 26, 31, 30, 27, 10, 32, 35, 15,
        3, 20, 20, 16, 27,  0, 14, 37, 12, 14,  5, 26,  6, 10, 15, 11, 33,
       30, 25, 25,  3, 36, 11, 12,  9, 25,  3, 25, 28, 21,  8, 11, 21, 11,
       37, 24, 32, 15, 20, 25, 17, 22, 11, 16,  1, 19,  0, 10, 29,  8, 13,
        7, 19, 23, 12,  6,  3, 33, 34,  9, 26, 18, 13,  2, 27, 34, 37, 31,
       25, 36, 28, 33, 13, 29, 35, 27, 30,  1,  6, 18,  7, 16, 34,  7, 22,
        5,  5,  8,  9, 37, 14,  5,  9,  0,  4, 26,  1,  3, 27,  5, 25, 22,
        7, 25, 26,  2, 37, 11, 27, 30, 16,  5, 35, 19, 33, 25,  2, 18, 25,
        6, 37,  6, 35, 34,  1, 33, 33, 14, 21, 12, 37, 33, 17, 24,  5,  7,
        0, 23, 25, 10, 35, 32,  9, 36, 18, 37, 23,  4, 36, 16, 27, 22, 32,
       35, 15, 11, 14,  5, 15, 25, 31, 34, 20, 11, 35, 32,  4, 11, 22, 21,
       11,  5,  8, 19, 17

In [ ]:
model.save('plant_diseases_model.h5')